## 2. Job Salary Prediction

 
<H4 align='center'> Fernanda Urrea, Rol:201551522-0, Matías Gómez, Rol: 201460501-3 


En esta sección se trabajará con el problema de predecir el salario que ofrece un anuncio en internet a través unicamente del texto del anuncio. El dataset es ofrecido por *Adzuna* como una competencia en la plataforma más grande de *data science* Kaggle, a través del siguiente __[link](https://www.kaggle.com/c/job-salary-prediction)__. El objetivo de la competencia, según *Adzuna*, es el de tener un motor que pueda predecir el salario de cualquier anuncio de trabajo en Reunio Unido, para poder mejorar la experiencia de los usuarios que buscan trabajos, ya que así pueden filtrar sin que el mismo empleador señale explícitamente cuánto paga.

<img src="http://s5047.pcdn.co/wp-content/uploads/2013/05/salary-prediction-engine-v2.png" title="Title text" width="50%"/>

La métrica de evaluación de la competencia es MAE (*mean absolute error*):

$$
L = \frac{1}{N} \sum_i^N  \mid y_i - \hat{y}_i \mid
$$


Para descargar los datos a utilizar (*Train_rev1*) debe estar registrado en la plataforma de Kaggle. Se cuenta con cientos de miles registros con textos sin procesar, es decir, no están estructurados.

#### a) Carge los datos csv de entrenamiento y cree un conjunto de validación con los últimos 10 mil datos en un dataframe de pandas. Describa los datos, apóyese de gráficos ¿Cuántos datos hay en cada conjunto? 

In [1]:
import pandas as pd
df = pd.read_csv("Train_rev1.csv")
df_train = df.iloc[:-10000]#Conjunto entrenamiento, todos los datos salvo los ultimos 10000
df_val = df.iloc[-10000:]#Ultimos 10000 datos
df.head()

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk


En el dataset se informan 12 características diferentes de cada una de las casas registradas en él. Estas son:
  
id: Identificador numérico para un anuncio.  
Title: Título del anuncio.  
FullDescription: Descripcion completa del anuncio.  
LocationRaw: Lugar detallado en el que se realizará el trabajo.  
LocationNormalized: Ciudad en la que se realizará el trabajo.  
ContractType: Tipo de contrato.  
ContractTime: Tiempo de contrato  
Company: Empresa que solicita el trabajo.  
Category: Categoria del trabajo asignada automaticamente por adzuna.  
SalaryRaw: Rango del salario.  
SalaryNormalized: Media de salario.  


In [2]:
df.columns

Index(['Id', 'Title', 'FullDescription', 'LocationRaw', 'LocationNormalized',
       'ContractType', 'ContractTime', 'Company', 'Category', 'SalaryRaw',
       'SalaryNormalized', 'SourceName'],
      dtype='object')

In [3]:
df.shape

(244768, 12)

El dataset contiene 244.768 registros distintos y 12 columnas que corresponden a los atributos.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 12 columns):
Id                    244768 non-null int64
Title                 244767 non-null object
FullDescription       244768 non-null object
LocationRaw           244768 non-null object
LocationNormalized    244768 non-null object
ContractType          65442 non-null object
ContractTime          180863 non-null object
Company               212338 non-null object
Category              244768 non-null object
SalaryRaw             244768 non-null object
SalaryNormalized      244768 non-null int64
SourceName            244767 non-null object
dtypes: int64(2), object(10)
memory usage: 22.4+ MB


Se muestra el resumen del dataframe 'df', en el cual no existen registros nulos para ninguna de las 12 características.
Además se informa el tipo de dato que presenta cada una de los atributos, destacando que la mayoría de ellos (10) son de texto y solo 2 numéricos

In [5]:
df.SalaryNormalized.describe()

count    244768.000000
mean      34122.577576
std       17640.543124
min        5000.000000
25%       21500.000000
50%       30000.000000
75%       42500.000000
max      200000.000000
Name: SalaryNormalized, dtype: float64

El método describe genera estadísticas descriptivas del dataframe en cuestión.  
Se informa el valor promedio, dispersión y la forma de la distribución de los datos según atributo (excluyendo los valores de NaN).

In [6]:
df_train.info() #Información del conjunto de entrenamiento

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234768 entries, 0 to 234767
Data columns (total 12 columns):
Id                    234768 non-null int64
Title                 234767 non-null object
FullDescription       234768 non-null object
LocationRaw           234768 non-null object
LocationNormalized    234768 non-null object
ContractType          62961 non-null object
ContractTime          172816 non-null object
Company               203031 non-null object
Category              234768 non-null object
SalaryRaw             234768 non-null object
SalaryNormalized      234768 non-null int64
SourceName            234767 non-null object
dtypes: int64(2), object(10)
memory usage: 21.5+ MB


In [8]:
df_val.info() #Información del conjunto de validación

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 234768 to 244767
Data columns (total 12 columns):
Id                    10000 non-null int64
Title                 10000 non-null object
FullDescription       10000 non-null object
LocationRaw           10000 non-null object
LocationNormalized    10000 non-null object
ContractType          2481 non-null object
ContractTime          8047 non-null object
Company               9307 non-null object
Category              10000 non-null object
SalaryRaw             10000 non-null object
SalaryNormalized      10000 non-null int64
SourceName            10000 non-null object
dtypes: int64(2), object(10)
memory usage: 937.6+ KB


#### b) Extraiga los datos de cada conjunto con los que trabajará, el *input* $X$, los textos, y el *output* $y$, los salarios.

In [7]:
text = df.FullDescription
salary = df.SalaryNormalized

#Extraen input y output del conjunto de entrenamiento
text_train = df_train.FullDescription
salary_train = df_train.SalaryNormalized


#Extraen input y output del conjunto de validación
text_val = df_val.FullDescription
salary_val = df_val.SalaryNormalized

#Algunos ejemplos
id = df.Id
for i in {1,2,3,4,5,6,7}:
    print ("INPUT NUMERO ", i, "\nTEXTO: \n", text[i], "\n OUTPUT NUMERO ", i, "\n SALARIO: ", salary[i], "\n \n \n")


INPUT NUMERO  1 
TEXTO: 
 Stress Engineer Glasgow Salary **** to **** We re currently looking for talented engineers to join our growing Glasgow team at a variety of levels. The roles are ideally suited to high calibre engineering graduates with any level of appropriate experience, so that we can give you the opportunity to use your technical skills to provide high quality input to our aerospace projects, spanning both aerostructures and aeroengines. In return, you can expect good career opportunities and the chance for advancement and personal and professional development, support while you gain Chartership and some opportunities to possibly travel or work in other offices, in or outside of the UK. The Requirements You will need to have a good engineering degree that includes structural analysis (such as aeronautical, mechanical, automotive, civil) with some experience in a professional engineering environment relevant to (but not limited to) the aerospace sector. You will need to dem

#### c) Realice un pre-procesamiento a los datos brutos de texto para extraer características y generar la representación de los datos de entrada al modelo $\vec{x}$. Comente sobre lo realizado.

In [9]:
text_train

0         Engineering Systems Analyst Dorking Surrey Sal...
1         Stress Engineer Glasgow Salary **** to **** We...
2         Mathematical Modeller / Simulation Analyst / O...
3         Engineering Systems Analyst / Mathematical Mod...
4         Pioneer, Miser  Engineering Systems Analyst Do...
5         Engineering Systems Analyst Water Industry Loc...
6         A globally renowned engineering and training c...
7         THIS IS A LIVE VACANCY NOT A GENERIC ADVERTISE...
8         This is an exceptional opportunity to join a c...
9         A subsea engineering company is looking for an...
10        Are you a successful, results driven person? A...
11        PROJECT ENGINEER PHAMACEUTICAL, c****  Excelle...
12        Aerospace Senior Composite Structures Engineer...
13        Senior Fatigue Static stress (metallic or comp...
14        A well respected restaurant located in the Der...
15        Our client are a leading supplier of precision...
16        A leading Subsea Engineering C

In [8]:
text_val

234768    Our Heathrow based client is urgently looking ...
234769    Production Planner Due to the rural location o...
234770    Temporary Assistant Accountant (****6 Months) ...
234771    Excellent Opportunity Available Commercial Ins...
234772    Title: Debt Management Team Leader My client i...
234773    Adecco are currently working in partnership wi...
234774    Expenses Administrator Godalming ****  benefit...
234775    My client is a true market leader within the s...
234776    Our client, a city based Lloyd s broker, is lo...
234777    Finance Assistant  Cash Book Salary Range  ***...
234778    Job title: OnLine Sales Manager / Call Centre ...
234779    FINANCE  ACCOUNTING OFFICER Central Delivery U...
234780    The Role Our clients, a Property and Investmen...
234781    Assisting in the production of monthly managem...
234782    Part Qualified  ACCA/CIMA or AAT Qualified is ...
234783    Secretary / Book keeper Shenley , near Radlett...
234784    Organisation Description: A we

Como el conjunto de entrenamiento es muy extenso, para el preprocesamiento de los datos, se genera un conjunto de entrenamiento más pequeño.


In [10]:
df_train = df.iloc[:-224267] #Nuevo conjunto de entrenamiento
df_val = df.iloc[-500:] #Nuevo conjunto de validación
text_train = df_train.FullDescription
salary_train = df_train.SalaryNormalized
text_val = df_val.FullDescription
salary_val = df_val.SalaryNormalized

In [63]:
text_train

0        Engineering Systems Analyst Dorking Surrey Sal...
1        Stress Engineer Glasgow Salary **** to **** We...
2        Mathematical Modeller / Simulation Analyst / O...
3        Engineering Systems Analyst / Mathematical Mod...
4        Pioneer, Miser  Engineering Systems Analyst Do...
5        Engineering Systems Analyst Water Industry Loc...
6        A globally renowned engineering and training c...
7        THIS IS A LIVE VACANCY NOT A GENERIC ADVERTISE...
8        This is an exceptional opportunity to join a c...
9        A subsea engineering company is looking for an...
10       Are you a successful, results driven person? A...
11       PROJECT ENGINEER PHAMACEUTICAL, c****  Excelle...
12       Aerospace Senior Composite Structures Engineer...
13       Senior Fatigue Static stress (metallic or comp...
14       A well respected restaurant located in the Der...
15       Our client are a leading supplier of precision...
16       A leading Subsea Engineering Company require a.

In [64]:
text_val

244268    Manual Machinist (Turning) Coventry **** to **...
244269    Procurement Planning Manager: A Procurement Pl...
244270    Job Title: Press Toolmaker Location: Coventry ...
244271    Assembly Technician / ElectroMechanical Fitter...
244272    Embedded Software Engineer – West Midlands  **...
244273    My client is currently on the lookout for a me...
244274    A leading international law firm is seeking a ...
244275    Leading Global Oil Services Company requires a...
244276    Senior Sales Consultant  Banking/Capital Marke...
244277    Business Analyst  ****  **** per day  3 months...
244278    Senior / Team Lead Java Developer REF **** Ede...
244279    As a Channel Sales professional working for a ...
244280    I have a **** year fixed rate contract open wi...
244281    An excellent opportunity to become the Head of...
244282    Position Big Data Scientist Location London Ty...
244283    We currently have a vacancy for a CAD Technici...
244284    IT Programme Delivery Manager 

A continuación, se realiza el preprocesamiento de los datos, basado en el código de la pregunta 1 resuelta en clases

In [14]:
import seaborn as sns

In [15]:
import numpy as np

In [16]:
import nltk
nltk.download('stopwords')
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Matu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Matu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Matu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [18]:
def word_extractor1(text):
    wordlemmatizer = WordNetLemmatizer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower())for word in word_tokenize(text) ]
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word
    return words

def word_extractor2(text):
    wordstemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordstemmer.stem(word.lower())for word in word_tokenize(text) ]
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word
    return words

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

texts_train1 = [word_extractor1(text) for text in text_train]
texts_val1 = [word_extractor1(text) for text in text_val]

vectorizer1 = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer1.fit(np.asarray(texts_train1))

features_train1 = vectorizer1.transform(texts_train1)
features_val1 = vectorizer1.transform(texts_val1)

texts_train2 = [word_extractor2(text) for text in text_train]
texts_val2 = [word_extractor2(text) for text in text_val]

vectorizer2 = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer2.fit(np.asarray(texts_train2))

features_train2 = vectorizer2.transform(texts_train2)
features_val2 = vectorizer2.transform(texts_val2)

In [20]:
texts_train1[0] #Ejemplo

' engineering system analyst dorking surrey salary ****k client located dorking , surrey looking engineering system analyst client provides specialist software development keywords mathematical modelling , risk analysis , system modelling , optimisation , miser , pioneeer engineering system analyst dorking surrey salary ****k'

In [72]:
print(features_train1)

  (0, 2470)	1
  (0, 2483)	1
  (0, 7607)	1
  (0, 10754)	1
  (0, 11505)	1
  (0, 12729)	1
  (0, 20251)	1
  (0, 21505)	1
  (0, 21646)	1
  (0, 22557)	1
  (0, 23481)	1
  (0, 23600)	1
  (0, 26177)	1
  (0, 27713)	1
  (0, 29275)	1
  (0, 31604)	1
  (0, 32229)	1
  (0, 34165)	1
  (0, 34501)	1
  (0, 35763)	1
  (0, 35993)	1
  (1, 1773)	1
  (1, 1842)	1
  (1, 1844)	1
  (1, 1846)	1
  :	:
  (20500, 21059)	1
  (20500, 22147)	1
  (20500, 22553)	1
  (20500, 24178)	1
  (20500, 26925)	1
  (20500, 27897)	1
  (20500, 28648)	1
  (20500, 29934)	1
  (20500, 30998)	1
  (20500, 31764)	1
  (20500, 32754)	1
  (20500, 32757)	1
  (20500, 32806)	1
  (20500, 32986)	1
  (20500, 35706)	1
  (20500, 36216)	1
  (20500, 36848)	1
  (20500, 37973)	1
  (20500, 39869)	1
  (20500, 40061)	1
  (20500, 40181)	1
  (20500, 40192)	1
  (20500, 40211)	1
  (20500, 40351)	1
  (20500, 40535)	1


In [73]:
vectorizer1

CountVectorizer(analyzer='word', binary='False', decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [13]:
vocab = vectorizer1.get_feature_names()
dist=list(np.array(features_train1.sum(axis=0)).reshape(-1,))
#for tag, count in zip(vocab, dist):
 #   print (count, tag)

NameError: name 'vectorizer1' is not defined

In [75]:
len(vocab)

40737

In [76]:
features_train1

<20501x40737 sparse matrix of type '<class 'numpy.int64'>'
	with 2132100 stored elements in Compressed Sparse Row format>

In [77]:
vocab = vectorizer1.get_feature_names()
dist=list(np.array(features_train2.sum(axis=0)).reshape(-1,))
lista = (vocab, dist)#Esta mal hecha la lista, hacer lista de tuplas
lista_ordenada = lista

In [82]:
for i in range(len(lista_ordenada[0])-2):
    if lista_ordenada[1][i]>5000:
        print(lista_ordenada[0][i],lista_ordenada[1][i])#Me las ingenié

abercorn 5262
abide 5188
aker 5312
andoid 8854
andrea 7539
appending 5045
authoritative 7142
bakewell 5175
brake 8766
brother 9039
chair 10618
chyflogaeth 7117
cilex 7922
clubhouse 5704
confirm 5989
consider 6738
cooks 5421
cs19 8940
diogelwch 5310
dispenser 5694
distributive 5828
doundation 5381
duggansearch 8775
ea 15463
essentialvirtualization 5164
faade 5296
fieldbased 5388
founded 5361
ggc 5602
gracing 8717
groupon 5060
hmpyoi 6778
homecaremedacs 6503
idoc 5419
incorporation 6562
infochasemedical 6113
interschool 9298
jackrabbit 10591
linuxadministratordevopsleadingmobileapplicionscompany_job 7497
louisea 6318
lydall 5793
mckinty 6229
methodolgy 9761
moran 5071
neonate 10954
nicetohave 6598
ohave 9175
paas 8414
pdr 11458
perfformiad 7871
pkcs 12207
postgresql 6724
programmeexcellent 8608
qtls 10423
residentialbased 7901
resulting 7635
rollins 11675
sara 14972
schedule 6199
selfesteem 5833
shawcarecleaningservices 8165
storyboarding 10088
string 16682
supermarket 5082


IndexError: list index out of range

In [2]:
vocab = vectorizer2.get_feature_names()
dist=list(np.array(features_train2.sum(axis=0)).reshape(-1,))
#for tag, count in zip(vocab, dist):
 #   print (count, tag)

NameError: name 'vectorizer2' is not defined

In [27]:
len(vocab)

3706

In [37]:
features_train2

<1x3706 sparse matrix of type '<class 'numpy.int64'>'
	with 21 stored elements in Compressed Sparse Row format>

In [29]:
vocab = vectorizer2.get_feature_names()
dist=list(np.array(features_train2.sum(axis=0)).reshape(-1,))
lista = (vocab, dist)
#lista.sort(key=lambda x: x[1],reverse=True)
lista_ordenada = lista

In [31]:
for i in range(len(lista_ordenada[0])-1):
    if lista_ordenada[1][i]>200:
        print(lista_ordenada[0][i],lista_ordenada[1][i])#Me las ingenié

appli 247
area 201
care 286
client 248
experi 362
job 326
look 282
offer 221
opportun 212
pleas 260
provid 243
recruit 233
requir 254
role 228
support 232
team 263
thi 277
train 223
within 289
work 401


En la celda anterior se muestran las palabras más frecuentes de cada representación generada

#### d) Intente resolver el problema enfrentándolo como regresión con el modelo de regresión lineal ordinaria en *sklearn*. ¿Qué es lo que hace *fit_intercept=True*? Evalúe la función objetivo (F.O.) utilizada y la métrica de la competencia (*mean absolute error*) en ambos conjuntos generados en el punto a). Comente lo observado.

In [21]:
#De aqui en adelante se utilizará la primera función (de las dos disponibles) para el preprocesamiento de los datos
print(texts_train1[0],salary_train[0])
for i in range(len(salary_train)-1):
    X_train=texts_train1[i]
    y_train=salary_train[i]
    model.fit(X_train,y_train)

 engineering system analyst dorking surrey salary ****k client located dorking , surrey looking engineering system analyst client provides specialist software development keywords mathematical modelling , risk analysis , system modelling , optimisation , miser , pioneeer engineering system analyst dorking surrey salary ****k 25000


NameError: name 'model' is not defined

In [83]:
from sklearn.linear_model import LinearRegression as LR
model = LR(fit_intercept=True, normalize=False)

X_train=features_train1
y_train=salary_train

X_val=features_val1
y_val=salary_val

model.fit(X_train,y_train)

... #measure F.O.

from sklearn.metrics import mean_absolute_error #measure MAE
print("MAE on train: ",mean_absolute_error(y_train, model.predict(X_train)))
print("MAE on validation: ",mean_absolute_error(y_val, model.predict(X_val)))

MAE on train:  75.85024136349291
MAE on validation:  137354.52200696507


#### e) Intente mejorar sus resultados en base a la métrica de la competencia (MAE) sobre el conjunto de validación. Comente sobre lo realizado.  *No se le pide que imite los resultados ganadores de la competencia (MAE de 3400), sino que mejore lo ya alcanzado siendo creativo.*
<div class="alert alert-warning"> HINT: Una opción es cambiar el *approach* de resolución desde regresión a casificación o trabajo sobre los datos (tal como limpiarlos).</div>

Hacer una mejor limpieza de los datos, es decir en el preprocesamiento comparar las palabras obtenidas despues de aplicar la función de lemmatizer con algun diccionario de palabras que se relacionen con el problema de modo que se quiten de los datos las palabras que no tienen sentido alguno.  
Otra forma de mejorar el problema es cambiar el enfoque en el que se esta tratando como por ejemplo ver el problema como un problema de vecino mas cercano mas que una regresión lineal donde se pierde información en la caracterización de las palabras.  
Otra forma de mejorar la resolución del problema es mediante el uso de distribuciones de probabilidad en las palabras de modo que el peso que se le asignan a las palabras este dado por la probabilidad de que esa palabra tenga cierta salario.  
